In [1]:
from datasets import load_metric, load_dataset, Audio
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch
from tqdm import tqdm
import json

In [2]:
# Hausa
from huggingface_hub import notebook_login
notebook_login()
cache_dir_cv_13 = "/data/users/kashrest/lrl-asr-experiments/data/cv_13"
data_cv = load_dataset("mozilla-foundation/common_voice_13_0", "ig", split="test", cache_dir=cache_dir_cv_13)

cache_dir_fleurs = "/data/users/kashrest/lrl-asr-experiments/data/fleurs"
data_fleurs = load_dataset("google/fleurs", "ig_ng", split="test", cache_dir=cache_dir_fleurs)

Found cached dataset common_voice_13_0 (/data/users/kashrest/lrl-asr-experiments/data/cv_13/mozilla-foundation___common_voice_13_0/ig/13.0.0/22809012aac1fc9803eaffc44122e4149043748e93933935d5ea19898587e4d7)
Found cached dataset fleurs (/data/users/kashrest/lrl-asr-experiments/data/fleurs/google___fleurs/ig_ng/2.0.0/af82dbec419a815084fa63ebd5d5a9f24a6e9acdf9887b9e3b8c6bbd64e0b7ac)


In [3]:
len(data_fleurs), len(data_cv)

(969, 4)

In [ ]:
data_fleurs

In [4]:
model_id = "facebook/mms-1b-all"
processor = AutoProcessor.from_pretrained(model_id)
model = Wav2Vec2ForCTC.from_pretrained(model_id)
processor.tokenizer.get_vocab()["ibo"]

{'!': 83,
 '"': 78,
 '$': 52,
 '%': 70,
 '&': 87,
 "'": 27,
 '+': 63,
 ',': 47,
 '-': 29,
 '.': 43,
 '/': 50,
 '0': 33,
 '1': 34,
 '2': 36,
 '3': 41,
 '4': 39,
 '5': 38,
 '6': 40,
 '7': 45,
 '8': 42,
 '9': 37,
 ':': 48,
 ';': 55,
 '</s>': 2,
 '<pad>': 0,
 '<s>': 1,
 '<unk>': 3,
 '[': 74,
 ']': 75,
 '_': 80,
 'a': 5,
 'b': 18,
 'c': 28,
 'd': 22,
 'e': 6,
 'f': 32,
 'g': 17,
 'h': 16,
 'i': 8,
 'j': 31,
 'k': 9,
 'l': 24,
 'm': 12,
 'n': 7,
 'o': 13,
 'p': 26,
 'q': 49,
 'r': 11,
 's': 23,
 't': 20,
 'u': 15,
 'v': 35,
 'w': 21,
 'x': 46,
 'y': 25,
 'z': 30,
 '|': 4,
 '£': 82,
 '¥': 69,
 '°': 68,
 '²': 88,
 'à': 57,
 'á': 67,
 'ã': 72,
 'ç': 61,
 'è': 56,
 'é': 65,
 'ì': 71,
 'í': 73,
 'ñ': 51,
 'ò': 54,
 'ó': 79,
 'õ': 64,
 'ù': 66,
 'ú': 81,
 'ü': 58,
 'ē': 85,
 'ł': 86,
 'ū': 77,
 '̀': 84,
 '̄': 53,
 'ṅ': 60,
 'ị': 19,
 'ọ': 14,
 'ụ': 10,
 '–': 59,
 '—': 76,
 '’': 44,
 '”': 62}

In [ ]:
data_cv[0]

In [ ]:
def _remove_non_alphanumeric_chars(transcription):
    remove_chars_list = ['!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', ':', ';', '?', '[', ']', '_', '|', '~', '£', '¥', '°', '²', '½', '¾', '–', '—', '‘', '’', '“', '”', '…']

In [5]:
#batch = processor.pad(input_features, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors="pt")
processor.tokenizer.set_target_lang("ibo")
model.load_adapter("ibo")
transcriptions = []
gold_transcriptions = []
for elem in tqdm(data_fleurs):
    inputs = processor(elem["audio"]["array"], sampling_rate=16_000, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs).logits
    ids = torch.argmax(outputs, dim=-1)[0]
    transcriptions.append(processor.decode(ids))
    gold_transcriptions.append(elem["raw_transcription"].lower())
    # 'wachambuzi wa soka wanamtaja mesi kama nyota hatari zaidi duniani'
    # => In English: "soccer analysts describe Messi as the most dangerous player in the world"

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 969/969 [1:04:55<00:00,  4.02s/it]


In [6]:
with open("facebook_mms-1b-all/zero-shot/fluers-test_output.jsonl", "w+") as f:
    for transcription in transcriptions:
        json.dump(transcription, f)
        f.write("\n")

In [7]:
with open("facebook_mms-1b-all/zero-shot/fluers-test_raw_lowercased.jsonl", "w+") as f:
    for transcription in gold_transcriptions:
        json.dump(transcription, f)
        f.write("\n")

In [8]:
len(transcriptions), len(gold_transcriptions)

(969, 969)

In [9]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
wer = wer_metric.compute(predictions=transcriptions, references=gold_transcriptions)
cer = cer_metric.compute(predictions=transcriptions, references=gold_transcriptions)
wer, cer

/tmp/ipykernel_5503/239588746.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


(0.5149128205128205, 0.1494831638435268)

In [ ]:
transcriptions[0]

In [ ]:
gold_transcriptions[0]